# Step 3 - Data Preprocessing

In the previous 2 modules:
* Learned to acquire data through different methods
* Discovered multiple formats in which data can be found and how to interact with each of them
* Performed exploratory data analysis 

Now we will give you a brief introduction on data preprocessing. By the end of this module you will be able to

1. Handle missing values and outliers
2. Perform data transformations
3. Apply feature selection techniques

For this module we are going to use a different dataset from the previous 2 modules. We are going to use the Credit Approval dataset from the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/). This data contains information regarding credit card applications. It is important to notice that the names of the features are not displayed in order to protect the confidentiality of the data. 

In [ ]:
# We start by loading the packages we are going to work with
from ucimlrepo import fetch_ucirepo 
import pandas as pd 
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
# load our data
credit_approval = fetch_ucirepo(id=27) 
  
# data (as pandas dataframes) 
X = credit_approval.data.features 
y = credit_approval.data.targets 
X

## Missing values and outliers

One of the most commonly encountered problems is the presence of missing values. Some of the ways we can threat these observations are the following:

* Remove observations: Simply remove the observations for which relevant data is missed. 
* Median imputations (continous features): Replace missing values with the median of the distribution, particullarly if the distribution of the data is skewed.
* Mean imputation (continous features): Replace missing values with the mean of the distribution, particullarly if the data is normally distributed.

Outliers represent cases in which a data point possess a value that significantly differs from the rest of the observations. One way to deal with these outliers, is to eliminate those data points in which the value is more than a certain number of standard deviations from the mean, or in other words, if the z-score of the observation is above a certain threshold.

Now, let's see some code to handle these cases.

In [ ]:
# First, we identify the presence of missing values
for x in X.columns:
    print(f"The column {x} has missing values: {X[x].isna().any()}")
print(f"\n{len(X)}")

In [ ]:
# Remove any row that has a missing value
X_clean = X.dropna() # This single line of code removes any row with a missing value in at least one column
for x in X_clean.columns:
    print(f"The column {x} has missing values: {X_clean[x].isna().any()}")
print(f"\n{len(X_clean)}")

As we see, the second version of our features dataframe has no missing values in any of the columns. However, we also lost 5% of the observations. 

In [ ]:
# Performing mean/median imputations
X_imp = X.copy() # For this example, we are going to work with a copy
floats = X_imp.select_dtypes(include=['float']) # We identify the columns with floating point values
print(f"Continous features: {floats.columns}\n")
for x in floats:
    if X_imp[x].isna().any():  # Check if the column has missing values
        mean = X_imp[x].mean() # Replace .mean() to impute the median
        X_imp[x].fillna(mean, inplace=True)
        
for x in X_imp.columns:
    print(f"The column {x} has missing values: {X_imp[x].isna().any()}")
print(f"\n{len(X_imp)}")

As we can see, none of the columns with continous features has missing values anymore. 

In [ ]:
# Handling outliers 

threshold = 3
valid_rows = [] # This list will store the rows with missing values

for x in floats:
    z_score = stats.zscore(X_clean[x])
    valid = abs(z_score) <= threshold
    valid_rows.append(valid)

# Combine the outlier masks for all columns
combined = pd.concat(valid_rows, axis=1).all(axis=1)
combined

# Remove rows with outliers
X_clean = X_clean[combined]
X_clean

## Data Transformations

Sometimes we need to transform our data in order to make it useful. Let's explore some common techniques to transform our data. 

## Dummies

For most of models, it is important to transform our categorical variables into a series binary columns (or dummies) to obtain a numerical representation of the presence or absence of each category. Let's see how to do that in the following code. 

In [ ]:
# Dummies
categorical = X.select_dtypes(include=['object']).columns# First, we identify the non-numerical features
X_dummies = pd.get_dummies(X[categorical], prefix=categorical) # Create dummies
X_dummies = pd.concat([X, X_dummies], axis=1) # Concatenate dataframes
X_dummies.drop(categorical, axis=1, inplace=True) # Keep only the dummies, as well as the original continous features

X_dummies

In this case, we have a boolean representation for each category.

## Interactions

For some models, like linear regression, the use of interactions between features might provide more information to the model and improve performance. 

In pandas, creating interaction is not a complicated task. 

In [ ]:
# Interaction

X_int = X.copy() 

X_int['A2*3'] = X_int['A2']*X_int['A3'] # We create an interaction feature of A2 and A3 by simply multiplying the columns.
X_int

## Data normalization and standardization

Another type of data transformation are the normalization and standardization of the continous features. These techniques help us reduce the impact of the outliers, as well as to improve the performance of our models.

* Normalization: In this method, we set all values between 0 and 1, with the minimum value assigned 0 and the maximum 1, with the rest of the values being transformed between 0 and 1.
* Standardization: In this method, all values are centered around the mean, with the mean receiving a value of 0. Different from the previous one, there is no default range in this method.

The *sklearn* library offers a convenient way to perform these tasks. 

In [ ]:
# Normalize continous features
X_norm = X.copy()  # We are going to work with a copy
X_floats = X[floats.columns]
scaler_minmax = MinMaxScaler()
X_norm[floats.columns] = scaler_minmax.fit_transform(X_floats)
X_norm

In [ ]:
# Standardize continous features
X_std = X.copy()  # We are going to work with a copy
X_floats = X[floats.columns]
scaler_standard = StandardScaler()
X_std[floats.columns] = scaler_standard.fit_transform(X_floats)
X_std

## Feature Selection

When we work with big datasets containing lots of features, it is important to only work with features that impact the prediction of our target. By doing this, we prevent overfitting our model, and we also use our computational resources more efficiently. The [sklearn feature selection](https://scikit-learn.org/stable/modules/feature_selection.html) module offers a series of feature selection methods which are convinient for this task. We will briefly explore some of them:

* Variance Threshold: This method estimates the probability for each feature of having a variance different from 0. We set a threshold for this probability and remove all features below that threshold.
* KBest: This method calculates a score (based on a function) between our features and our target, and select the *k* number of features that scored the highest. 



In [ ]:
# Variance Threshold
X_floats = X_std.copy()
X_floats = X_floats[floats.columns]
for x in floats: # Our X_std still had some missing values. We are going to imput the mean. 
    if X_floats[x].isna().any():  # Check if the column has missing values
        mean = X_floats[x].mean() # Replace .mean() to impute the median
        X_floats[x].fillna(mean, inplace=True)
selector = VarianceThreshold(threshold=0.982) # We can adjust the threshold. We are using an arbitrary high threshold for demonstrative purposes.
X_var = selector.fit_transform(X_floats)
selected_columns = X_floats.columns[selector.get_support()]
X_var = pd.DataFrame(X_var, columns=selected_columns)
print(X_var)

As we see, our features dataframe now has less features. In our threshold, we asked to remove all features with a probability (p-value) of having a variance equal to 0 of 1.82% or higher (which is a very strick threshold).

In [ ]:
# K-Best
y_bin = np.where(y["A16"]=="+",1,0) # The current dtype of y is object. We need to convert it into a binary series.
k_selector = SelectKBest(f_classif, k=2) # K = 2 because we only want to keep 2 features.
X_k = k_selector.fit_transform(X_floats, y_bin)
k_columns = X_floats.columns[k_selector.get_support()]
X_k = pd.DataFrame(X_k, columns=k_columns)
X_k

The two features with the highest scores using the F-statistic as our scoring method, are A8 and A3. Therefore, the method returns a dataframe with those 2 columns.

We invite you to take a look at more methods trough the official documentation.

## Dimensionality Reduction

Another helpful way to improve the performance of our models and take better advantage of our computational resources is by performing dimensionality reduction to our data. This technique implies the transformation of our data into a lower dimension space, while performing the most relevant information. A very common approach for this task is the [Principal Component Analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis). 

The next cell shows the implementation of a code using *sklearn*. 

In [ ]:
# Principal Component Analysis
X_pca = X_floats.copy()

pca = PCA(n_components=2) # We are going to reduce our data into a 2D space. 
X_pca = pca.fit_transform(X_pca)
X_pca = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(2)])
print(X_pca)

As we see, our data was transformed and reduced into two components.